In [1]:
# Ref: https://qiita.com/noca/items/00646efd9d4a7302f522
from ipywidgets import Textarea
import io
if 'open' in globals():
    del open
if 'input' in globals():
    del input
original_open = open
class custom_open():
    def __init__(self):
        self.text = ''
    def __call__(self, file, *args, **kwargs):
        if file == 0:
            return io.StringIO(self.text)
        return original_open(file, *args, **kwargs)
    def updater(self, change):
        self.text = change["new"]
class custom_input():
    def __init__(self):
        self.__sio = io.StringIO('')
        self.shell = get_ipython()
        if self.shell.events.callbacks['pre_run_cell'] != []:
            self.shell.events.callbacks['pre_run_cell'] = []
        self.shell.events.register('pre_run_cell', self.pre_run_cell)
    def __call__(self):
        return self.__sio.readline().strip()
    def pre_run_cell(self, info):
        text = self.shell.user_ns.get('text_area', None).value
        self.__sio = io.StringIO(text)
open = custom_open()
input = custom_input()
text_area = Textarea()
text_area.observe(open.updater, names='value')
display(text_area)

Textarea(value='')

In [68]:
# セグメント木で管理する関数
# モノイドある必要がある
def segfunc(x, y):
    return max(x,y)

class SegTree:
    # N: 要素数, INF: 初期値
    def __init__(self, segfunc, N: int, INF: int):
        # セグメント木の要素数
        n0 = 1 << ((N-1).bit_length())
        self.offset = n0
        self.data = [INF] * (n0 << 1)
        
        self.segfunc = segfunc
        self.INF = INF
    
    # インデックス pos の値を x に更新
    # pos: 0-indexed
    # x: 更新する値
    def update(self, pos, x):
        # 実際のデータは最下段のインデックス [n0,2*n0) に入る
        # pos に対応するセグメント木の要素のインデックスは pos + self.offset
        pos += self.offset
        # 更新処理
        self.data[pos] = x
        # 親を更新していく
        while pos > 1:
            # 左シフトして親のインデックスを求める
            pos >>= 1
            # pos の子に対して segfunc を適用
            self.data[pos] = self.segfunc(self.data[pos<<1], self.data[pos<<1 | 1])
    
    # 区間 [l,r) に segfunc を適用する処理
    # l, r: 0-indexed
    def query(self, l, r):
        result = self.INF
        # 最下段の要素のインデックス
        l = l + self.offset
        r = r + self.offset
        # 最下段の左と右の外側から、親を辿って範囲を狭めながら segfunc を適用していく
        while l < r:
            if l & 1:
                # 左端のブロックを調べる
                result = self.segfunc(result, self.data[l])
                # 右のブロックへ
                l += 1
            if r & 1:
                # 左のブロックを調べる
                result = self.segfunc(result, self.data[r-1])
            # 親ブロックへ移動
            l >>= 1
            r >>= 1
        
        return result

N,Q=map(int,input().split())
Queries=[list(map(int,input().split())) for _ in range(Q)]

sg = SegTree(segfunc, N, 0)

for q,a,b in Queries:
    a-=1
    if q==1:
        sg.update(a,b)
    else:
        b-=1
        res=sg.query(a,b)
        print(res)

0
13
